In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

In [3]:
from ml.model import U_Net

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
import nibabel as nib
def load_data_pair(path_to_folder):
    path_to_head = path_to_folder + "/head.nii.gz"
    path_to_vessels = path_to_folder + "/vessels.nii.gz"
    
    head_file = nib.load(path_to_head)
    vessels_file = nib.load(path_to_vessels)
    
    head_vol = np.array(head_file.dataobj, dtype=np.float32)
    vessels_vol = np.array(vessels_file.dataobj, dtype=np.float32)
    return(head_vol, vessels_vol)

In [6]:
#head_vol, vessels_vol = load_data_pair("/home/msst/Documents/medtech/brain_seg_dataset/CT_S5020")

In [7]:
head_vol = np.random.rand(32, 32, 32).astype(np.float32)
vessels_vol = np.random.rand(32, 32, 32).astype(np.float32)
brain_vol = np.random.rand(32, 32, 32).astype(np.float32)

In [8]:
head_vol = torch.tensor(head_vol).unsqueeze(0).unsqueeze(0)
vessels_vol = torch.tensor(vessels_vol).unsqueeze(0).unsqueeze(0)
brain_vol = torch.tensor(brain_vol).unsqueeze(0).unsqueeze(0)
print(head_vol.shape)
print(vessels_vol.shape)
print(vessels_vol.shape)


torch.Size([1, 1, 32, 32, 32])
torch.Size([1, 1, 32, 32, 32])
torch.Size([1, 1, 32, 32, 32])


In [9]:
model = U_Net()

In [11]:
a = model(head_vol)
a.shape

torch.Size([1, 64, 32, 32, 32])

In [10]:
def get_total_params(model):
    total_params = sum(
    param.numel() for param in model.parameters()
    )
    return(total_params)

get_total_params(model)

103536449

In [11]:
loss = nn.BCELoss(reduction='mean')
optim = torch.optim.Adam(model.parameters(), lr=3e-4, betas=(0.9, 0.999), eps=1e-06, weight_decay=0.0)

In [12]:
for epoch in range(300):
    optim.zero_grad()
    out = model(head_vol)
    out = out[0]
    loss_train = loss(out, vessels_vol.type(torch.float))
    print(loss_train)
    loss_train.backward()
    optim.step()

ValueError: Using a target size (torch.Size([1, 1, 32, 32, 32])) that is different to the input size (torch.Size([1, 64, 32, 32, 32])) is deprecated. Please ensure they have the same size.